# Poisson Equation in Parallel

NGSolve can be executed on a cluster using the MPI message passing interface.
You can download [poisson_mpi.py](poisson_mpi.py) and run it as

> mpirun -np 4 python3 poisson_mpi.py 

This computes and saves a solution which 
we can visualize it with [drawsolution.py](drawsolution.py)

> netgen drawsolution.py

For proper parallel execution, Netgen/NGSolve must be configured with
`-DUSE_MPI=ON`. Recent binaries for Linux and Mac are built with MPI support (?). If you are unsure, when  your Netgen/NGSolve supports MPI, look for outputs like "Including MPI version 3.1" during Netgen startup.

## MPI-parallel execution using ipyparallel

For the MPI jupyter-tutorials we use `ipyparallel` module. Please consult https://ipyparallel.readthedocs.io for installation.

On my notebook I have created the profile once by the command

>ipython profile create --parallel --profile=mpi

Since I have only two physical cores, I edited the file
`.ipython/profile_mpi/ipcluster_config.py` to allow for oversubscription:

> c.MPILauncher.mpi_args = ['--oversubscribe']



I start the cluster via

> ipcluster start --engines=MPI -n 4 --profile=mpi




In jupyter, we can then connect to the cluster via a `Client` object of ipyparallel. 

In [1]:
from ipyparallel import Client
c = Client(profile='mpi')
c.ids

Waiting for connection file: ~/.ipython/profile_mpi/security/ipcontroller-client.json


OSError: Connection file '~/.ipython/profile_mpi/security/ipcontroller-client.json' not found.
You have attempted to connect to an IPython Cluster but no Controller could be found.
Please double-check your configuration and ensure that a cluster is running.

In [2]:
help (Client)

Help on class Client in module ipyparallel.client.client:

class Client(traitlets.traitlets.HasTraits)
 |  Client(*args, **kw)
 |  
 |  A semi-synchronous client to an IPython parallel cluster
 |  
 |  Parameters
 |  ----------
 |  
 |  url_file : str
 |      The path to ipcontroller-client.json.
 |      This JSON file should contain all the information needed to connect to a cluster,
 |      and is likely the only argument needed.
 |      Connection information for the Hub's registration.  If a json connector
 |      file is given, then likely no further configuration is necessary.
 |      [Default: use profile]
 |  profile : bytes
 |      The name of the Cluster profile to be used to find connector information.
 |      If run from an IPython application, the default profile will be the same
 |      as the running application, otherwise it will be 'default'.
 |  cluster_id : str
 |      String id to added to runtime files, to prevent name collisions when using
 |      multiple cluster

We use mpi4py https://mpi4py.readthedocs.io/ for issuing MPI calls from Python. The %%px syntax magic causes parallel execution of that cell:

In [ ]:
%%px 
from mpi4py import MPI
comm = MPI.COMM_WORLD
print (comm.rank, comm.size)

The *master* process (rank==0) generates the mesh, and distributes it within the group of processes defined by the *communicator*. All other ranks receive a part of the mesh. The function mesh.GetNE(VOL) returns the local number of elements:

In [ ]:
%%px
from ngsolve import *
from netgen.geom2d import unit_square

if comm.rank == 0:
    mesh = Mesh(unit_square.GenerateMesh(maxh=0.1).Distribute(comm))
else:
    mesh = Mesh(netgen.meshing.Mesh.Receive(comm))
print (mesh.GetNE(VOL))

We can define spaces, bilinear / linear forms, and gridfunctions in the same way as in sequential mode. But now, the degrees of freedom are distributed on the cluster following the distribution of the mesh. The finite element spaces define how the dofs match together.

In [ ]:
%%px
fes = H1(mesh, order=3, dirichlet=".*")
u,v = fes.TnT()

a = BilinearForm(grad(u)*grad(v)*dx)
pre = Preconditioner(a, "local")
a.Assemble()

f = LinearForm(1*v*dx).Assemble()
gfu = GridFunction(fes)

from ngsolve.krylovspace import CGSolver
inv = CGSolver(a.mat, pre.mat, printing=comm.rank==0, maxsteps=200, tol=1e-8)
gfu.vec.data = inv*f.vec

Parallel pickling allows to serialize the distributed solution and transfer it to the client:

In [ ]:
%%px
netgen.meshing.SetParallelPickling(True)

In the next bit of code, all processes of the client object `c` pickle their local `gfu` object. Setting `SetParallelPickling(True)` enables communication so that the process with rank=0 gets the whole mesh and the computed solution. 

In [ ]:
gfu = c[:]["gfu"]

We can now draw the whole solution using the the master process's `gfu[0]`.

In [ ]:
from ngsolve.webgui import Draw
Draw (gfu[0])

Drawing `gfu[n]` will draw only part of the solution that the process with rank=`n` possesses. 

In [ ]:
Draw (gfu[3])

We can also visualize the sub-domains obtained by the automatic partitioning, without using any computed solution, as follows.

In [ ]:
%%px
fesL2 = L2(mesh, order=0)
gfL2 = GridFunction(fesL2)
gfL2.vec.local_vec[:] = comm.rank

In [ ]:
gfL2 = c[:]["gfL2"]
Draw (gfL2[0])